In [454]:
from IPython.core.debugger import set_trace
import numpy as np
import sys
#import collections
import re
import pprint
pp = pprint.PrettyPrinter(indent=1)
f=open('day24input.txt') #not with read because thats probably the whole file
lines = [line.rstrip('\n') for line in f]

In [455]:
def initialize(boost):
    f=[]
    type = 'immune'
    id = 0
    for line in lines[1:]:
        #print (line)
        if line == "":
            pass
        elif line =='Infection:':
            type = 'infection'
            id=0
        else:
            #print(line)
            id+=1
            if '(' in line:
                b= re.search('(.*) units each with (.*) hit points \((.*)\) with an attack that does (.*) (.*) damage at initiative (.*)',line)
            else:
                b= re.search('(.*) units each with (.*) hit points w(.*)th an attack that does (.*) (.*) damage at initiative (.*)',line)
            group ={'id':type+str(id),
                   'alliance':type,
                   'units':int(b[1]),
                   'hp':int(b[2]),
                   'attackpower':int(b[4]),
                    'weak':[],
                    'immune':[],
                   'attacktype':b[5],
                    'initiative':int(b[6]),
                    'target':None,
                    'istarget':False,
                    'ingame':True

                   }
            group['ep']=group['attackpower']*group['units']

            if len(b[3])>2:
                specials = b[3].split('; ')
                for item in specials:
                    specialresult = re.search('(.*) to (.*)',item)
                    if specialresult[1]=='weak':
                        group['weak']=specialresult[2].split(', ')
                    elif specialresult[1]=='immune':
                        group['immune']=specialresult[2].split(', ')
                    else:
                        print ('specialresult[1]')
                        sys.exit()
            if group['alliance']=='immune':
                group['attackpower']+=boost
            group['ep']=group['attackpower']*group['units']
            f.append(group)        
            #pp.pprint(group)

    return (f)

In [456]:
def checkend(f):
    imm=[group for group in f if group['alliance']=='immune']
    inf=[group for group in f if group['alliance']=='infection']
    if (len(imm)==0) or (len(inf))==0:
        print ('battle-end')
        print ('immune system',sum([x['units'] for x in imm]))
        print ('infection',sum([x['units'] for x in inf]))
        if  sum([x['units'] for x in imm])>0:
            print ('winner')
            print (f)
            sys.exit()
        return True
    else: return False
    
def update_ep(group):
    pass
    #group['attackpower']*group['units']


def select_target(group,potentialtargets):# returns a groups ID to attack or none if none is selected
    #print (potentialtargets)
    for p in potentialtargets:
        p['potentialdamage']=calc_damage(group,p)
    #potentialdamage, effective power; if there is still a tie, it chooses the defending group with the highest initiative. 
    potentialtargets.sort(key=lambda x:(x['potentialdamage'],x['ep'],x['initiative']),reverse=True)
    
    #if target exists with nonnegative potential damage, return that target
    #print (potentialtargets)
    if potentialtargets[0]['potentialdamage']>0:
        #print ('attacker {}, potentialdamage {}, potentialtargetid {}'.format(group['id'],potentialtargets[0]['potentialdamage'],potentialtargets[0]['id']))

        return potentialtargets[0]['id']
    else:
        #print ('no target found',[[x['id'],x['potentialdamage'],x['ep'],x['initiative']] for x in potentialtargets])
        return None
    
    
def select_targets(f):
    f.sort(key=lambda x:(x['ep'],x['initiative']),reverse=True)
    #print ('selecting targets',[[x['id'],x['ep'],x['initiative']] for x in f])
    for group in f:
        #select enemy army that is from other alliance and is not a target already and is in game
        #print ('selecting target for ',group['id'])
        potentialtargets = [potential for potential in f if (potential['alliance']!=group['alliance'])and (potential['ingame']) and (potential['istarget'] is False)]
        if len(potentialtargets)==0:
            #print ('no potential targets\n',group)
            pass
        else:
            group['target'] = select_target(group,potentialtargets)
            #print ('target found ', group['target'])
            if group['target']!=None:
                for g in f:
                    if g['id'] == group['target']:
                        g['istarget'] = True 
    return f

def deal_damage(f):
    f.sort(key=lambda x:(x['initiative']),reverse=True)
    #print ('selecting target\n',[[x['id'],x['initiative']] for x in f])
    for group in f:
        if group['ingame'] and group['target'] is not False:
            for g in f:
                if g['id'] == group['target']:
                    #target found it's g
                    killed = calc_damage(group,g)//g['hp']
                    #print (group['id'],'killed ',killed, 'from',g['id'])
                    g['units']-= killed
                    if g['units']<=0:
                        g['ingame']=False
                        #print (g['id'],' is dead')
                    else:
                        g['ep']=g['attackpower']*g['units']
                        #print (g['id'],' new ep ',g['ep'])
    return f

def removedeadandresettargets(f):
    #print ('removedead',[[x['id'],x['ingame'],x['units'] ]for x in f])
    f= [group for group in f if group['ingame']]
    for group in f:
        group['istarget']=False
        group['target']=False
    #print ('units remaining ',len(f))
    #if len(f)==4:
    #    #print (f)
    return f
        
    
def fight(f):
    #print ('start fight')
    f=select_targets(f)
    f=deal_damage(f)
    f=removedeadandresettargets(f)
    if checkend(f):
        print ('end')
        return(True,f)
    else:##
        return (False,f)

def calc_damage(attacker,defender):
    if attacker['attacktype'] in defender['weak']:
        return attacker['ep']*2
    elif attacker['attacktype'] in defender['immune']:
        return 0
    else:#
        return attacker['ep']
    
def combat(f):
    fightready = False
    i=0
    while fightready is False:
        result =fight(f) 
        fightready=result[0]
        f=result[1]
        i+=1
        if i>10000:
            print (f)
            print (len(f))
            print ('units',[[x['id'],x['units'],x['attacktype'],x['immune']]for x in f])
            sys.exit(0)
    return 

In [461]:
boostmax = 40
for x in range(36,boostmax):
    print ('current boost',x)
    f=initialize(x)
    result = combat(f)
    pass

current boost 36
battle-end
immune system 9174
infection 0
winner
[{'id': 'immune2', 'alliance': 'immune', 'units': 210, 'hp': 5535, 'attackpower': 91, 'weak': ['bludgeoning'], 'immune': [], 'attacktype': 'slashing', 'initiative': 20, 'target': False, 'istarget': False, 'ingame': True, 'ep': 19110, 'potentialdamage': 63}, {'id': 'immune8', 'alliance': 'immune', 'units': 958, 'hp': 10067, 'attackpower': 65, 'weak': ['bludgeoning'], 'immune': [], 'attacktype': 'cold', 'initiative': 13, 'target': False, 'istarget': False, 'ingame': True, 'ep': 62270, 'potentialdamage': 63}, {'id': 'immune10', 'alliance': 'immune', 'units': 1111, 'hp': 3593, 'attackpower': 45, 'weak': [], 'immune': [], 'attacktype': 'bludgeoning', 'initiative': 11, 'target': False, 'istarget': False, 'ingame': True, 'ep': 49995, 'potentialdamage': 63}, {'id': 'immune4', 'alliance': 'immune', 'units': 42, 'hp': 4951, 'attackpower': 396, 'weak': [], 'immune': ['cold'], 'attacktype': 'fire', 'initiative': 9, 'target': False, 

SystemExit: 

C:\Users\Jesse\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
a=True
not a is False

In [383]:
27
a=[{'id': 'infection7', 'alliance': 'infection', 'units': 62, 'hp': 44733, 'attackpower': 258, 'weak': ['radiation', 'fire'], 'immune': ['cold', 'bludgeoning'], 'attacktype': 'cold', 'initiative': 18, 'target': False, 'istarget': False, 'ingame': True, 'ep': 15996, 'potentialdamage': 0}, {'id': 'infection3', 'alliance': 'infection', 'units': 1113, 'hp': 41570, 'attackpower': 63, 'weak': [], 'immune': ['fire', 'bludgeoning'], 'attacktype': 'fire', 'initiative': 14, 'target': False, 'istarget': False, 'ingame': True, 'ep': 70119, 'potentialdamage': 0}, {'id': 'immune5', 'alliance': 'immune', 'units': 6019, 'hp': 6574, 'attackpower': 35, 'weak': [], 'immune': ['cold', 'slashing', 'fire'], 'attacktype': 'bludgeoning', 'initiative': 4, 'target': False, 'istarget': False, 'ingame': True, 'ep': 210665, 'potentialdamage': 768}, {'id': 'infection10', 'alliance': 'infection', 'units': 32, 'hp': 36170, 'attackpower': 24, 'weak': ['slashing', 'cold'], 'immune': ['bludgeoning'], 'attacktype': 'radiation', 'initiative': 1, 'target': False, 'istarget': False, 'ingame': True, 'ep': 768, 'potentialdamage': 0}]

In [384]:
pp.pprint(a)

[{'alliance': 'infection',
  'attackpower': 258,
  'attacktype': 'cold',
  'ep': 15996,
  'hp': 44733,
  'id': 'infection7',
  'immune': ['cold', 'bludgeoning'],
  'ingame': True,
  'initiative': 18,
  'istarget': False,
  'potentialdamage': 0,
  'target': False,
  'units': 62,
  'weak': ['radiation', 'fire']},
 {'alliance': 'infection',
  'attackpower': 63,
  'attacktype': 'fire',
  'ep': 70119,
  'hp': 41570,
  'id': 'infection3',
  'immune': ['fire', 'bludgeoning'],
  'ingame': True,
  'initiative': 14,
  'istarget': False,
  'potentialdamage': 0,
  'target': False,
  'units': 1113,
  'weak': []},
 {'alliance': 'immune',
  'attackpower': 35,
  'attacktype': 'bludgeoning',
  'ep': 210665,
  'hp': 6574,
  'id': 'immune5',
  'immune': ['cold', 'slashing', 'fire'],
  'ingame': True,
  'initiative': 4,
  'istarget': False,
  'potentialdamage': 768,
  'target': False,
  'units': 6019,
  'weak': []},
 {'alliance': 'infection',
  'attackpower': 24,
  'attacktype': 'radiation',
  'ep': 768,


In [ ]:
current boost 37
battle-end
immune system 8826
infection 0
winner
too low

In [ ]:
units': 6275, bij boost 29
maar infinite loop

28 wordt immune langzaam gesloopt nog

In [379]:
test = [1,2,3]
test.sort(reverse=True)

In [380]:
test

[3, 2, 1]

In [411]:
1288+569+6969

8826